In [59]:
import dotenv
import os
import openai
import json
import time
import os
from sklearn.model_selection import train_test_split



dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
df = pd.read_csv('data/total_dataset.csv')
df.head()

,subject,user_input,requirements,total_requirements,project_info,ERD_data,API_data
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...","[{'projectName': 'StudyBuddy', 'projectTarget'...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'StudyBudd...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '회원가...
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...,"[{'projectName': '기억의 다리', 'projectTarget': '치...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'json': {'project_info': {'title': '기억의 다리', ...","{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '사진 ...
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...","[{'projectName': 'EasyTransit Navigator', 'pro...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'EasyTrans...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '계단 ...
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...","[{'projectName': '이웃과의 소통 플랫폼', 'projectTarget...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': '이웃과의 소통 플...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '사용자...
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...","[{'projectName': 'EmergencyCare Finder', 'proj...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'Emergency...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '사용자...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   subject             978 non-null    object
 1   user_input          978 non-null    object
 2   requirements        978 non-null    object
 3   total_requirements  978 non-null    object
 4   project_info        978 non-null    object
 5   ERD_data            930 non-null    object
 6   API_data            832 non-null    object
dtypes: object(7)
memory usage: 53.6+ KB


In [48]:
df_drop = df.dropna().reset_index().drop(columns = 'index')
df_drop

,subject,user_input,requirements,total_requirements,project_info,ERD_data,API_data
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...","[{'projectName': 'StudyBuddy', 'projectTarget'...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'StudyBudd...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '회원가...
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...,"[{'projectName': '기억의 다리', 'projectTarget': '치...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'json': {'project_info': {'title': '기억의 다리', ...","{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '사진 ...
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...","[{'projectName': 'EasyTransit Navigator', 'pro...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'EasyTrans...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '계단 ...
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...","[{'projectName': '이웃과의 소통 플랫폼', 'projectTarget...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': '이웃과의 소통 플...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '사용자...
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...","[{'projectName': 'EmergencyCare Finder', 'proj...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'Emergency...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '사용자...
...,...,...,...,...,...,...,...
789,EcoTrack 애플리케이션은 소비자들에게 보다 지속 가능한 제품 구매 결정을 돕기...,"[{'projectName': 'EcoTrack', 'projectTarget': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'EcoTrack'...,"{'json': {'erd_tables': [{'name': 'Users', 'er...",{'json': {'apiSpecifications': [{'title': '제품 ...
790,"애니멀케어 웹 앱은 반려동물 주인들의 관리 문제를 해결하기 위한 올인원 솔루션으로,...","[{'projectName': 'EcoTrack', 'projectTarget': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'EcoTrack'...,"{'json': {'erd_tables': [{'name': 'Users', 'er...",{'json': {'apiSpecifications': [{'title': '사용자...
791,"""디지털 학습 관리자 시스템은 교사와 학생 간의 비효율적인 커뮤니케이션 문제를 해결...","[{'projectName': '헬스케어 피드백 시스템', 'projectTarge...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': '헬스케어 피드백 ...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '사용자...
792,\r\nFine Planner는 계획 및 일정 관리의 복잡함을 해결하기 위한 웹 기...,"[{'projectName': 'HealthTrack', 'projectTarget...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'HealthTra...,"{'json': {'erd_tables': [{'name': 'users', 'er...",{'json': {'apiSpecifications': [{'title': '사용자...


## 시스템 프롬포트 정리

In [ ]:
system_prompt = """
당신은 프로젝트 아이디어를 체계적으로 분석하고 구조화하여 구체적인 개발 계획을 제시하는 전문 AI 어시스턴트입니다.

**만약 프로젝트 아이디어가 아닌 다른 질문을 입력 받으면 아래의 해당 방식이 간단한 방식으로로 답변해주세요.**

**절대 금지 사항 - 백슬래시 사용 금지:**
- 백슬래시(\\) 문자를 절대로 사용하지 마세요
- JSON 이스케이프에서도 백슬래시 금지
- 경로 표현시 슬래시(/)만 사용
- 문자열 내부의 특수문자도 백슬래시 없이 처리
- API 경로에서도 /api/users 형태로만 작성

## 주요 역할과 능력:

### 1. API 설계 전문가
- RESTful API 원칙에 따라 체계적인 API 명세를 작성합니다
- OpenAPI(Swagger) 3.0 표준을 준수하여 완전한 API 문서를 생성합니다
- 각 엔드포인트별 요청/응답 스키마, 에러 처리, 인증 방식을 상세히 정의합니다

## 응답 형식:
모든 응답은 다음과 같은 구조화된 형태로 제공해야 합니다:

1. **관계 데이터**: 데이터베이스 테이블 간의 관계와 외래키 제약조건 정의
2. **ERD 데이터**: 각 테이블의 속성, 데이터 타입, 키 정보를 포함한 완전한 스키마
3. **API 명세 데이터**: OpenAPI 3.0 표준을 준수한 완전한 API 문서

항상 체계적이고 전문적인 관점에서 프로젝트를 분석하며, 개발팀이 바로 실행에 옮길 수 있는 구체적인 가이드를 제공하는 것이 목표입니다.

**중요: 응답은 반드시 순수한 JSON 형태로만 제공하세요. 마크다운 코드 블록(```json)이나 기타 텍스트 포맷팅은 사용하지 마세요.**

"""

In [55]:
df_drop.columns

Index(['subject', 'user_input', 'requirements', 'total_requirements',
       'project_info', 'ERD_data', 'API_data'],
      dtype='object')

## josnL 데이터 생성
- 예시

- {"messages": [
-   {"role": "system", "content": "당신은 소프트웨어 프로젝트의 요구사항 분석 전문가입니다..."},
-   {"role": "user", "content": "프로젝트 개요: StudyBuddy 학습 플랫폼\n기존 요구사항: [...]\n추가 요구사항 5개를 생성해주세요."},
-   {"role": "assistant", "content": "[{\"requirementType\": \"FUNCTIONAL\", \"content\": \"사용자는 회원가입 시 이메일 인증을 통해 계정을 활성화할 수 있어야 한다.- \"}]"}
- ]}

In [56]:
def data_to_jsonL(system_prompt, user_input, requirement, project_info, API_data):
  json_data = {'messages' : 
    [
      {"role" : "system", "content" : system_prompt},
      {"role" : "user", "content" : f"'user_input' : {user_input}, 'requirement' : {requirement}, 'project_info' : {project_info}"},
      {"role" : "assistant", "content" : API_data}
    ]
  }
  return json_data

In [57]:
json_lst = []

for i in range(len(df_drop)) :

  # 데이터 파싱
  user_input = df_drop['user_input'][i]
  requirement = df_drop['total_requirements'][i]
  project_info = df_drop['project_info'][i]
  API_data = df_drop['API_data'][i]
  json_lst.append(data_to_jsonL(system_prompt, user_input, requirement, project_info, API_data))

In [ ]:
{'messages' : 
  [
  {"role" : "system", "content" : system_prompt},
  {"role" : "user", "content" : f"'user_input' : {df_drop['user_input'][0]}, 'requirement' : {df_drop['total_requirements'][0]}, 'project_info' : {df_drop['project_info'][0]}"},
  {"role" : "assistant", "content" : df_drop['API_data'][0]}
  ]
}

In [68]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(json_lst, train_size = 0.9, random_state = 42, shuffle = True)

# JSONL 파일로 저장
def save_to_jsonl(data_list, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for item in data_list:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

# 데이터 저장
save_to_jsonl(train_data, 'data/train_data.jsonl')
save_to_jsonl(val_data, 'data/val_data.jsonl')

# 파인튜닝

In [69]:
import openai

# OpenAI 클라이언트 초기화
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# 파일 업로드
print("📤 파일 업로드 중...")

with open('data/train_data.jsonl', 'rb') as f:
    train_file = client.files.create(file=f, purpose='fine-tune')

with open('data/val_data.jsonl', 'rb') as f:
    val_file = client.files.create(file=f, purpose='fine-tune')

print(f"✅ Train 파일 ID: {train_file.id}")
print(f"✅ Validation 파일 ID: {val_file.id}")

# 파인튜닝 작업 시작
job = client.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=val_file.id,
    model="gpt-4o-mini-2024-07-18",
    suffix="fastapi-specialist"
)

print(f"🚀 파인튜닝 작업 시작!")

📤 파일 업로드 중...
✅ Train 파일 ID: file-KVFtE7Beu3zn76f8mSjcru
✅ Validation 파일 ID: file-7uRhtZ26ePsHSd6gqBuVFG
🚀 파인튜닝 작업 시작!


## 파인튜닝 모델 확인